<a href="https://colab.research.google.com/github/barileao/SVM_Scania/blob/master/svm_scania.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando packages

In [1]:
!pip install pyod
!pip install impyute

     |████████████████████████████████| 92kB 2.6MB/s 
  Created wheel for pyod: filename=pyod-0.7.5-cp36-none-any.whl size=96597 sha256=4f726b9c12e47f6bca2e782e70a2dae324d0a5e15ed64db692cdecb5fe29961c
  Stored in directory: /root/.cache/pip/wheels/35/bc/f4/2cef321eac851ba02f533e3921b296bb4ce4998c8a4c9a8d6e
Successfully built pyod


# Imports

In [2]:
# importing all the necessary libraries
from google.colab import files
import io
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.impute import SimpleImputer,MissingIndicator
from sklearn.decomposition import TruncatedSVD
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.metrics import confusion_matrix,f1_score,log_loss
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import missingno as msno
from sklearn.decomposition import PCA
from datetime import datetime
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.feature_bagging import FeatureBagging
from fancyimpute import SoftImpute
from impyute.imputation.cs import mice



Using TensorFlow backend.


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


# Acessar os dados no Drive


In [3]:
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /drive


# Upload dos dados


In [4]:
filetrain = r'/drive/My Drive/IFES/TCC_Scania/Codes/aps_failure_training_set.csv'
filetest = r'/drive/My Drive/IFES/TCC_Scania/Codes/aps_failure_test_set.csv'

train = pd.read_csv(filetrain)
print('Number of data points in the train dataset', train.shape[0])
print('Number of features in the given train dataset', train.shape[1])
test = pd.read_csv(filetest)
print('Number of data points in the test dataset', test.shape[0])
print('Number of features in the given test dataset', test.shape[1])

Number of data points in the train dataset 60000
Number of features in the given train dataset 171
Number of data points in the test dataset 16000
Number of features in the given test dataset 171


In [5]:
# ----------------------Train--------------------------------

# Replacing all the class labels as follows:

# neg = 0
# pos = 1
class_labels_train = []
for x in tqdm(train.index):        # tqdm = barra de progresso
    if train['class'].loc[x] == 'neg':
        class_labels_train.append(0)
        
    else:
        class_labels_train.append(1)
        

train['class'] = class_labels_train

# Distrubution of class labels in train dataset
train['class'].value_counts()




100%|██████████| 60000/60000 [00:02<00:00, 26383.47it/s]


0    59000
1     1000
Name: class, dtype: int64

# Histograma

In [0]:
# Histogram plot of class labels of train dataset
plt.hist(train['class'])
plt.title("Histogram plot of class labels")
print("Percentage of negative class in the given dataset :", (train[train['class']== 0].shape[0] / train.shape[0]) * 100)
print("Percentage of positive class in the given dataset :", (train[train['class']== 1].shape[0] / train.shape[0]) * 100)


# Remover e substituir NAN e NA

In [6]:
# ----------------------Train--------------------------------
# Replacing the na and nan values with np.NaN
train.replace(to_replace='nan', value = np.NaN,inplace = True)

train.replace(to_replace = 'na', value = np.NaN, inplace = True)

columns_names = train.columns

train.describe(include='all')

# Dropping all the columns where the number of missing values are more than 42K (>70%)
train.dropna(axis = 1, thresh=42000,inplace= True)
train.head()



,class,aa_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,ag_004,ag_005,ag_006,ag_007,ag_008,ag_009,ah_000,ai_000,aj_000,ak_000,al_000,am_0,an_000,ao_000,ap_000,aq_000,ar_000,as_000,at_000,au_000,av_000,ax_000,ay_000,ay_001,ay_002,ay_003,ay_004,ay_005,ay_006,ay_007,...,db_000,dc_000,dd_000,de_000,df_000,dg_000,dh_000,di_000,dj_000,dk_000,dl_000,dm_000,dn_000,do_000,dp_000,dq_000,dr_000,ds_000,dt_000,du_000,dv_000,dx_000,dy_000,dz_000,ea_000,eb_000,ec_00,ed_000,ee_000,ee_001,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,0,76698,2130706438,280,0,0,0,0,0,0,37250,1432864,3664156,1007684,25896,0,2551696,0,0,0,0,0,4933296,3655166,1766008,1132040,0,0,0,0,1012,268,0,0,0,0,0,469014,4239660,703300,...,18,5330690,4732,1126,0,0,0,0,0,0,0,0,62282,85908,32790,0,0,202710,37928,14745580,1876644,0,0,0,0,2801180,2445.8,2712,965866,1706908,1240520,493384,721044,469792,339156,157956,73224,0,0,0
1,0,33058,0,NaN,0,0,0,0,0,0,18254,653294,1720800,516724,31642,0,1393352,0,68,0,0,0,2560898,2127150,1084598,338544,0,0,0,0,0,0,0,0,0,0,0,71510,772720,1996924,...,NaN,NaN,3312,522,0,0,0,0,0,0,0,0,33736,36946,5936,0,0,103330,16254,4510080,868538,0,0,0,0,3477820,2211.76,2334,664504,824154,421400,178064,293306,245416,133654,81140,97576,1500,0,0
2,0,41040,228,100,0,0,0,0,0,0,1648,370592,1883374,292936,12016,0,1234132,0,0,0,0,0,2371990,2173634,300796,153698,0,0,0,0,358,110,0,0,0,0,0,0,870456,239798,...,0,2341048,1494,152,0,0,0,0,0,0,0,0,13876,38182,8138,0,0,65772,10534,300240,48028,0,0,0,0,1040120,1018.64,1020,262032,453378,277378,159812,423992,409564,320746,158022,95128,514,0,0
3,0,12,70,66,0,10,0,0,0,318,2212,3232,1872,0,0,0,2668,0,0,0,642,3894,10184,7554,10764,1014,0,0,0,0,60,6,0,0,0,0,0,0,0,2038,...,0,2578,76,62,0,0,0,0,0,0,0,0,232,0,0,2014,370,48,18,15740,1822,20174,44,0,0,0,1.08,54,5670,1566,240,46,58,44,10,0,0,0,4,32
4,0,60874,1368,458,0,0,0,0,0,0,43752,1966618,1800340,131646,4588,0,1974038,0,226,0,0,0,3230626,2618878,1058136,551022,0,0,0,0,1788,642,0,0,0,0,42124,372236,2128914,819596,...,0,3590004,2026,444,0,0,0,0,0,0,0,0,44946,62648,11506,0,0,149474,35154,457040,80482,98334,27588,0,0,21173050,1116.06,1176,404740,904230,622012,229790,405298,347188,286954,311560,433954,1218,0,0


# Train data: mediana, média e mais frequente 

In [7]:
# using median
impute_median = SimpleImputer(missing_values= np.nan, strategy='median',copy = True, verbose= 2)
train_median = pd.DataFrame(impute_median.fit_transform(train),columns=train.columns)
train_median['class'].value_counts()
#train_imputed_median.to_csv("Train_imputed_median")

# using mean
#impute_mean = SimpleImputer(missing_values= np.nan, strategy='mean',copy = True, verbose= 2)
#train_mean = pd.DataFrame(impute_mean.fit_transform(train),columns=train.columns)
#train_imputed_mean.to_csv("Train_imputed_mean")

# using most frequent
#impute_most_frequent = SimpleImputer(missing_values= np.nan, strategy='most_frequent',copy = True, verbose= 2)
#train_most_frequent = pd.DataFrame(impute_most_frequent.fit_transform(train),columns=train.columns)
#train_imputed_most_frequent.to_csv("Train_imputed_most_frequent")

# Feature engineering, creating a new feature set which indicates 
# the missing value in a given row and column as True else False
#missing_impute = MissingIndicator()
#miss = missing_impute.fit_transform(train)
#train_miss_indi = pd.DataFrame(miss)
#train_miss_indi.to_csv('train_miss_indi.csv')

0.0    59000
1.0     1000
Name: class, dtype: int64

# Retirar os Outliers para o treinamento

In [8]:
#coletando os outliers do train para 'trainol'
trainol_median = train_median.loc[(train_median['class'] ==1)]
#print(trainol_median)

#removendo os outliers para treinamento 
train_median = train_median[train_median['class'] != 1]
print(train_median)
train_median['class'].value_counts()

       class    aa_000        ac_000  ...    ee_009  ef_000  eg_000
0        0.0   76698.0  2.130706e+09  ...       0.0     0.0     0.0
1        0.0   33058.0  0.000000e+00  ...    1500.0     0.0     0.0
2        0.0   41040.0  2.280000e+02  ...     514.0     0.0     0.0
3        0.0      12.0  7.000000e+01  ...       0.0     4.0    32.0
4        0.0   60874.0  1.368000e+03  ...    1218.0     0.0     0.0
...      ...       ...           ...  ...       ...     ...     ...
59995    0.0  153002.0  6.640000e+02  ...   28588.0     0.0     0.0
59996    0.0    2286.0  2.130707e+09  ...       0.0     0.0     0.0
59997    0.0     112.0  2.130706e+09  ...       0.0     0.0     0.0
59998    0.0   80292.0  2.130706e+09  ...  388422.0     0.0     0.0
59999    0.0   40222.0  6.980000e+02  ...     158.0     0.0     0.0

[59000 rows x 161 columns]


0.0    59000
Name: class, dtype: int64

# Padronizando os dados

In [0]:
# Get column names first
names = train.columns

# Create the Scaler object
scaler = preprocessing.StandardScaler()

# Fit your data on the scaler object - train_imputed_median
train_median = scaler.fit_transform(train_median)
train_median = pd.DataFrame(train_median, columns=names)
trainol_median = scaler.fit_transform(trainol_median)
trainol_median = pd.DataFrame(trainol_median, columns=names)

# Fit your data on the scaler object - train_imputed_mean
#train_mean = scaler.fit_transform(train_mean)
#train_mean = pd.DataFrame(train_mean, columns=names)

# Fit your data on the scaler object - train_imputed_most_frequent
#train_most_frequent = scaler.fit_transform(train_most_frequent)
#train_most_frequent = pd.DataFrame(train_most_frequent, columns=names)


# Retirar a coluna ['class'] do train e trainol

In [0]:
train_median = train_median.drop(['class'], axis =1)
trainol_median = trainol_median.drop(['class'], axis =1)

# Training: One-Class SVM

In [48]:
oneclass = svm.OneClassSVM(kernel='rbf',verbose=True)

oneclass.fit(train_median)
#oneclass.fit(train_mean)
#oneclass.fit(train_most_frequent)

[LibSVM]

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto_deprecated',
            kernel='rbf', max_iter=-1, nu=0.5, random_state=None,
            shrinking=True, tol=0.001, verbose=True)

# Salvar arquivo com Pickle

In [0]:
import pickle

# salvar arquivo de treino
filename = 'finalized_model_median.sav'
pickle.dump(oneclass, open(filename, 'wb'))



In [0]:
import pickle

# carregar arquivo de treino
loaded_model = pickle.load(open('/drive/My Drive/IFES/TCC_Scania/finalized_model_median_woclass.sav', 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

# Predict

In [24]:
teste = trainol_median

#pred = oneclass.predict(teste)
pred_teste = loaded_model.predict(teste)

#print('pred teste = ', pred_teste)

# tirado de: https://scikit-learn.org/stable/auto_examples/svm/plot_oneclass.html
n_error_test = pred_teste[pred_teste == -1].size
print('erro teste = ', n_error_test)



erro train =  29500
erro teste =  1000
